In [46]:
import json
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer

%config InlineBackend.figure_format = 'retina';

In [6]:
df = pd.read_json("JEOPARDY_QUESTIONS1.json")

In [7]:
df

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [33]:
categories = set([i.lower() for i in df['category']])
categories = list(categories)
categories

['"u" have the right',
 'name the author',
 '"ten" & up',
 'bring your "a" game',
 'characters in dickens books',
 'portuguys',
 'justin time',
 'modern dance',
 'road trip usa',
 '"p"eople',
 'presents',
 'sci-fi movies by character',
 'unusual animals',
 'the "first"',
 'historic geography',
 '6-letter words',
 'national geographic kids',
 'prospero',
 'foreign wars',
 'weapons of the old west',
 'surfing the oed',
 'meet the parents',
 'summer school  at yale',
 '"c" in shakespeare',
 '"i"s in the prize',
 'holy cross',
 'wonders of the midwest',
 'todd mcfarlane figurines',
 'special ops movies',
 'make-up test',
 "let's go work out",
 'commencement speeches',
 'economies of the world',
 'skin care',
 'i love new york',
 'they all played hamlet',
 'obsolete definitions',
 'caan',
 'indianagrams',
 'papal people',
 'j.c.',
 'time to retire',
 '"we" the people',
 'hit the library',
 'mark my words',
 'political lingo origins',
 'mountain country',
 'they came from another world',
 "t

In [30]:
len(categories)

27916

In [32]:
len(list(set([i for i in df.category])))

27995

In [45]:
categories = [re.sub('[^a-zA-Z]', ' ', cat) for cat in categories]
categories = [re.sub(r'\s+', ' ', cat) for cat in categories]
categories = [c.strip() for c in categories]

In [52]:
category = [nltk.sent_tokenize(c) for c in categories] 
# for c in categories:
#     category.append(nltk.sent_tokenize(c))

all_words = [nltk.word_tokenize(c) for cat in category for c in cat]

# for cat in category:
#     for c in cat:
#         nltk.word_tokenize(c)

# Removing Stop Words
# from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

In [53]:
all_words

[['u', 'right'],
 ['name', 'author'],
 ['ten'],
 ['bring', 'game'],
 ['characters', 'dickens', 'books'],
 ['portuguys'],
 ['justin', 'time'],
 ['modern', 'dance'],
 ['road', 'trip', 'usa'],
 ['p', 'eople'],
 ['presents'],
 ['sci', 'fi', 'movies', 'character'],
 ['unusual', 'animals'],
 ['first'],
 ['historic', 'geography'],
 ['letter', 'words'],
 ['national', 'geographic', 'kids'],
 ['prospero'],
 ['foreign', 'wars'],
 ['weapons', 'old', 'west'],
 ['surfing', 'oed'],
 ['meet', 'parents'],
 ['summer', 'school', 'yale'],
 ['c', 'shakespeare'],
 ['prize'],
 ['holy', 'cross'],
 ['wonders', 'midwest'],
 ['todd', 'mcfarlane', 'figurines'],
 ['special', 'ops', 'movies'],
 ['make', 'test'],
 ['let', 'go', 'work'],
 ['commencement', 'speeches'],
 ['economies', 'world'],
 ['skin', 'care'],
 ['love', 'new', 'york'],
 ['played', 'hamlet'],
 ['obsolete', 'definitions'],
 ['caan'],
 ['indianagrams'],
 ['papal', 'people'],
 ['j', 'c'],
 ['time', 'retire'],
 ['people'],
 ['hit', 'library'],
 ['mark', 

In [54]:
word2vec = Word2Vec(all_words, min_count=2)

In [56]:
vocabulary = word2vec.wv
print(vocabulary)

In [60]:
vocabulary.most_similar('history')

[('tv', 0.9092385768890381),
 ('movie', 0.9085818529129028),
 ('american', 0.9066474437713623),
 ('world', 0.9064135551452637),
 ('names', 0.9060899615287781),
 ('u', 0.9017610549926758),
 ('film', 0.8983402848243713),
 ('movies', 0.896399736404419),
 ('name', 0.8940685391426086),
 ('old', 0.8905946612358093)]

In [61]:
vocabulary.most_similar('american')

[('names', 0.9161056280136108),
 ('words', 0.9099653959274292),
 ('sports', 0.9099439978599548),
 ('history', 0.9066473841667175),
 ('state', 0.904628336429596),
 ('film', 0.9043441414833069),
 ('world', 0.9040049910545349),
 ('movie', 0.9031883478164673),
 ('national', 0.9019918441772461),
 ('movies', 0.896014392375946)]

In [63]:
annoy_index = AnnoyIndexer(word2vec, 100)
# Derive the vector for the word "science" in our model
vector = wv["history"]
# The instance of AnnoyIndexer we just created is passed
approximate_neighbors = vocabulary.most_similar([vector], topn=11, indexer=annoy_index)
# Neatly print the approximate_neighbors and their corresponding cosine similarity values
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

normal_neighbors = vocabulary.most_similar([vector], topn=11)
print("\nExact Neighbors")
for neighbor in normal_neighbors:
    print(neighbor)

Approximate Neighbors
('quoth', 0.4230474829673767)
('punk', 0.4187002182006836)
('older', 0.41392403841018677)
('sinatra', 0.4102816581726074)
('pad', 0.40727341175079346)
('vessels', 0.4013880491256714)
('gas', 0.3991371989250183)
('dental', 0.3982469439506531)
('roger', 0.3974401354789734)
('hoosier', 0.39597398042678833)
('niro', 0.3959692120552063)

Exact Neighbors
('file', 0.3708578050136566)
('quoth', 0.3342515528202057)
('punk', 0.32418113946914673)
('older', 0.31302982568740845)
('sinatra', 0.30446454882621765)
('pad', 0.29735034704208374)
('acting', 0.28920578956604004)
('portmanteau', 0.28653353452682495)
('vessels', 0.2833274304866791)
('disguise', 0.28298524022102356)
('gas', 0.2779277265071869)
